<a href="https://colab.research.google.com/github/DS-Dalen22/DedSec-Dalen/blob/main/Training_Experiment_CV_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Начало работы

**Экспериментальный ноутбук**

In [ ]:
import zipfile
import os
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp /content/drive/MyDrive/Citypersons.v9i.yolov8.zip .
!unzip Citypersons.v9i.yolov8.zip

In [ ]:
zip_file = "/content/Citypersons.v9i.yolov8.zip"
zip_path = "/content/Citypersons.v9i.yolov8"

In [ ]:
#Распаковка
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(zip_path)

print(f"Zip file '{zip_file}' has been extracted to '{zip_path}'")

In [ ]:
#Вывод датасета по файлам и количества содержимых в нем изображений
for root, dirs, files in os.walk(zip_path):
  print(f"Папка: {root}, Файлов: {len(files)}")

In [ ]:
#Создание переменных для файлов с изображениями и разметками уже разделенных на три выборки
train_image_folder = '/content/Citypersons.v9i.yolov8/train/images/'
train_label_folder = '/content/Citypersons.v9i.yolov8/train/labels/'

val_image_folder = '/content/Citypersons.v9i.yolov8/valid/images/'
val_label_folder = '/content/Citypersons.v9i.yolov8/valid/labels/'

test_image_folder = '/content/Citypersons.v9i.yolov8/test/images/'
test_label_folder = '/content/Citypersons.v9i.yolov8/test/labels/'

In [ ]:
#Пример вывода изображения и его метки
train_images = os.listdir(train_image_folder)
train_labels = os.listdir(train_label_folder)

image_name = train_images[0]
label_name = image_name.replace('.jpg', '.txt')

print(f"Изображение: {image_name}")
print(f"Аннотация: {label_name}")

with open(os.path.join(train_label_folder, label_name), 'r') as label_file:
    annotations = label_file.readlines()
    for annotation in annotations:
        print(annotation.strip())

In [ ]:
!pip install albumentations
import os
import cv2
import numpy as np
import random
from shutil import copyfile
from PIL import Image
from albumentations import (
    Compose, OneOf, HorizontalFlip, VerticalFlip, Rotate, RandomBrightnessContrast, CLAHE, HueSaturationValue, RandomCrop, BboxParams
)

from albumentations import Compose
import json

In [ ]:
# Подготовительный этап, агументация данных библиотекой albumentations

aug = Compose([
    HorizontalFlip(p=0.5),
    RandomBrightnessContrast(p=0.3),
    HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5)
], bbox_params=BboxParams(format='yolo', min_visibility=0.1))


def augment_single_image(img_path, label_path, output_img_dir, output_label_dir):
    """
    Функция для аугментации одного изображения и соответствующих ему разметок (боксов).
    - img_path: путь к исходному изображению
    - label_path: путь к файлу с разметкой (YOLO формат)
    - output_img_dir: куда сохранить аугментированное изображение
    - output_label_dir: куда сохранить обновлённую разметку
    """

    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    with open(label_path, 'r') as f:
        lines = f.readlines()

    bboxes = []
    for line in lines:
        class_id, x_center, y_center, width, height = map(float, line.split())
        bboxes.append([x_center, y_center, width, height])

    if bboxes:
        augmented = aug(image=img, bboxes=bboxes)
        aug_img = augmented['image']
        aug_bboxes = augmented['bboxes']
    else:
        aug_img = img
        aug_bboxes = []

    os.makedirs(output_img_dir, exist_ok=True)
    os.makedirs(output_label_dir, exist_ok=True)

    output_img_path = os.path.join(output_img_dir, os.path.basename(img_path))
    cv2.imwrite(output_img_path, cv2.cvtColor(aug_img, cv2.COLOR_RGB2BGR))

    output_label_path = os.path.join(output_label_dir, os.path.basename(label_path))
    with open(output_label_path, 'w') as f:
        for bbox in aug_bboxes:
            f.write(f"0 {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n")  # class_id=0 (person)


input_image_dir = "/content/Citypersons.v9i.yolov8/train/images/"
input_label_dir = "/content/Citypersons.v9i.yolov8/train/labels/"
output_image_dir = "/content/train/augmented/images"
output_label_dir = "/content/train/augmented/labels"

for img_name in os.listdir(input_image_dir):
    if img_name.endswith('.jpg'):
        img_path = os.path.join(input_image_dir, img_name)
        label_path = os.path.join(input_label_dir, img_name.replace('.jpg', '.txt'))

        if os.path.exists(label_path):
            augment_single_image(img_path, label_path, output_image_dir, output_label_dir)

print("Аугментация завершена")


In [ ]:
# Пример вывода изображения
from IPython.display import Image, display
display(Image(os.path.join(output_image_dir, os.listdir(output_image_dir)[0])))

In [ ]:
from google.colab.patches import cv2_imshow

def draw_bboxes(image_path, label_path):

    img = cv2.imread(image_path)
    h, w = img.shape[:2]


    with open(label_path, 'r') as f:
        lines = f.readlines()


    for line in lines:
        class_id, x_center, y_center, width, height = map(float, line.split())
        x1 = int((x_center - width/2) * w)
        y1 = int((y_center - height/2) * h)
        x2 = int((x_center + width/2) * w)
        y2 = int((y_center + height/2) * h)


        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)


    cv2_imshow(img)
    return img

original_img_name = os.listdir("/content/Citypersons.v9i.yolov8/train/images")[0]
original_img_path = os.path.join("/content/Citypersons.v9i.yolov8/train/images", original_img_name)
original_label_path = os.path.join("/content/Citypersons.v9i.yolov8/train/labels", original_img_name.replace('.jpg', '.txt'))


augmented_img_path = os.path.join("/content/train/augmented/images", original_img_name)
augmented_label_path = os.path.join("/content/train/augmented/labels", original_img_name.replace('.jpg', '.txt'))

print("Оригинальное изображение:")
original_img = draw_bboxes(original_img_path, original_label_path)

print("\nАугментированное изображение:")
if os.path.exists(augmented_img_path):
    augmented_img = draw_bboxes(augmented_img_path, augmented_label_path)
else:
    print("Ошибка")

In [ ]:
from IPython.display import display, Markdown
import os

train_images = os.listdir("/content/Citypersons.v9i.yolov8/train/images")[:5]
display(Markdown("**Train images:** " + ", ".join(train_images)))

with open(f"/content/Citypersons.v9i.yolov8/train/labels/{train_images[1].replace('.jpg', '.txt')}", "r") as f:
    display(Markdown("**Пример аннотации:**\n```\n" + f.read() + "\n```"))

In [ ]:
!pip install ultralytics
from ultralytics import YOLO

* Первый запуск обучения. Использовалась архитектура Yolo nano, оптимизатор по умолчанию, weight_decay - чтобы исключить переобучение. Также кастомные настройки гиперпараметров Yolo

In [ ]:
model = YOLO('yolov8n.pt')
model.train(
    data='data.yaml',
    epochs=100,
    batch=8,
    lr0=0.01,
    lrf=0.1,
    optimizer='AdamW',
    weight_decay=0.0005,
    hsv_h=0.2,
    hsv_s=0.7,
    flipud=0.3,
    fliplr=0.5,
    name='yolo_custom_aug'
)



Вывод графиков

In [ ]:
from PIL import Image
Image.open('runs/detect/yolo_custom_aug/results.png')

* При выводе метрик recall который отвечает за то чтобы модель находила людей равен всего 50% - много пропусков.

Точность определения человека (Precision) - 70%

mAP@50 0.60 - общая точность нормальная
mAP@50-95 0.36 - рамки довольно неточные

In [ ]:
model = YOLO('runs/detect/yolo_custom_aug/weights/best.pt')

metrics = model.val()

print("Результаты валидации:")
print(f"Precision всех классов: {metrics.box.p.mean():.2f}")
print(f"Recall всех классов: {metrics.box.r.mean():.2f}")
print(f"mAP@50: {metrics.box.map50:.2f}")
print(f"mAP@50-95: {metrics.box.map:.2f}")

* Второй запуск обучения. Модель nano

оптимизатор SGD

Batch 16

In [ ]:
SAVE_PATH = '/content/drive/MyDrive/yolo_training'

model = YOLO('yolov8n.pt')
model.train(
    data='data.yaml',
    epochs=100,
    batch=16,
    lr0=0.001,
    lrf=0.1,
    optimizer='SGD',
    weight_decay=0.0005,
    hsv_h=0.2,
    hsv_s=0.7,
    fliplr=0.5,
    name='yolo_custom_aug2',
    project=SAVE_PATH
)


In [ ]:
Image.open('/content/drive/MyDrive/yolo_training/yolo_custom_aug2/results.png')

Результаты практически идентичны с первым запуском

In [ ]:
model = YOLO('/content/drive/MyDrive/yolo_training/yolo_custom_aug2/weights/best.pt')


metrics = model.val()


print("Результаты валидации:")
print(f"Precision всех классов: {metrics.box.p.mean():.2f}")
print(f"Recall всех классов: {metrics.box.r.mean():.2f}")
print(f"mAP@50: {metrics.box.map50:.2f}")
print(f"mAP@50-95: {metrics.box.map:.2f}")

* Третий запуск обучения.

Сталкиваюсь с проблемой ограниченных ресурсов Google Collab

Изучив вариации размеров архитектур YOLO перехожу на (M)

Batch выставлен на 4 чтобы попробовать ускорить процесс поскольку imgsize установлен на 960. На тот момент также думал что это позволит увеличить точность

Оптимизатор по умолчанию YoloV8

In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8m.pt')

model.train(
    data='data.yaml',
    epochs=40,
    batch=4,
    imgsz=960,
    lr0=0.003,
    lrf=0.1,
    optimizer='AdamW',
    weight_decay=0.001,
    hsv_h=0.1,
    hsv_s=0.7,
    fliplr=0.5,
    mosaic=1.0,
    patience=20,
    project='runs/detect',
    name='yolov8m_custom960',
    exist_ok=True,
    resume=False
)

# Второй пайплайн

* Проанализировав прошлую работу я решаю проверить свой датасет а также провожу консультацию после которой понимаю что входящих изображений слишком мало и они слишком однообразны. Тажке полностью переделываю аугментацию

* К основному датасету я добавляю еще один новый что суммарно увеличило мои изображения вдвое а также новый датасет содержал разнообразие что должно было исправить проблему слабого Recall

In [ ]:
import zipfile
import os
import cv2
from google.colab import files
from shutil import copy2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp /content/drive/MyDrive/human.v1i.yolov8.zip .
!cp /content/drive/MyDrive/Citypersons.v9i.yolov8.zip .

In [ ]:
!unzip -o Citypersons.v9i.yolov8.zip -d /content/citypersons
!unzip -o human.v1i.yolov8.zip -d /content/human_dataset

*Функция комбинирования датасетов*

* Скрипт создает структуру combined_dataset и собирает туда картинки/лейблы из двух источников с префиксами, чтобы избежать коллизий имён.

* ensure_test_split — если нет test, создаёт его, перемещая ratio часть train в test.

* check_labels — быстрая валидация аннотаций: формат строк, общее число боксов, и подсчёт «очень маленьких» боксов.

# Изменения №1

*обратная связь по проекту*

Что изменилось?
* функция для комбинирования датасета была перепроверена на наличие ошибок с лейблами и вторым датасетом. Поскольку структура датасета human отличалась отсутствием test выборки, функция def ensure_test_split создает test/images test/labels папки путем случайного поднабора данных из train - 15%.

* Функция merge_datasets для каждого изображения копирует его в целевую папку tgt_img с новой именной prefix_originalname (например city_img001.jpg), чтобы избежать коллизий имён при мердже разных наборов.




In [ ]:
import os, glob, random, shutil
from shutil import copy2


for split in ["train", "valid", "test"]:
    os.makedirs(f"/content/combined_dataset/{split}/images", exist_ok=True)
    os.makedirs(f"/content/combined_dataset/{split}/labels", exist_ok=True)


def ensure_test_split(base_dir, ratio=0.15):
    train_img_dir = os.path.join(base_dir, "train", "images")
    train_lbl_dir = os.path.join(base_dir, "train", "labels")
    test_img_dir = os.path.join(base_dir, "test", "images")
    test_lbl_dir = os.path.join(base_dir, "test", "labels")

    if os.path.exists(test_img_dir):
        print(f"[INFO] test уже существует в {base_dir}")
        return

    os.makedirs(test_img_dir, exist_ok=True)
    os.makedirs(test_lbl_dir, exist_ok=True)

    all_imgs = glob.glob(os.path.join(train_img_dir, "*.*"))
    test_size = int(len(all_imgs) * ratio)
    test_imgs = random.sample(all_imgs, test_size)

    for img_path in test_imgs:
        fname = os.path.basename(img_path)
        base = os.path.splitext(fname)[0]
        lbl_path = os.path.join(train_lbl_dir, base + ".txt")

        shutil.move(img_path, os.path.join(test_img_dir, fname))
        if os.path.exists(lbl_path):
            shutil.move(lbl_path, os.path.join(test_lbl_dir, base + ".txt"))

    print(f"[INFO] Создали test в {base_dir}: {test_size} файлов")


def merge_datasets(src_img, src_lbl, tgt_img, tgt_lbl, prefix):
    if not os.path.exists(src_img):
        print(f"[WARN] Папки {src_img} нет — пропускаем")
        return

    valid_exts = ('.jpg', '.jpeg', '.png', '.JPG')
    for fname in os.listdir(src_img):
        if fname.lower().endswith(valid_exts):
            base = os.path.splitext(fname)[0]
            src_img_path = os.path.join(src_img, fname)
            src_lbl_path = os.path.join(src_lbl, base + ".txt")

            new_img_name = f"{prefix}_{fname}"
            new_lbl_name = f"{prefix}_{base}.txt"

            copy2(src_img_path, os.path.join(tgt_img, new_img_name))
            if os.path.exists(src_lbl_path):
                copy2(src_lbl_path, os.path.join(tgt_lbl, new_lbl_name))
            else:
                open(os.path.join(tgt_lbl, new_lbl_name), "w").close()
                print(f"[INFO] У {fname} нет лейбла → создали пустой .txt")


ensure_test_split("/content/human_dataset", ratio=0.15)

# citypersons
merge_datasets('/content/citypersons/train/images', '/content/citypersons/train/labels',
               '/content/combined_dataset/train/images', '/content/combined_dataset/train/labels', prefix="city")
merge_datasets('/content/citypersons/valid/images', '/content/citypersons/valid/labels',
               '/content/combined_dataset/valid/images', '/content/combined_dataset/valid/labels', prefix="city")
merge_datasets('/content/citypersons/test/images', '/content/citypersons/test/labels',
               '/content/combined_dataset/test/images', '/content/combined_dataset/test/labels', prefix="city")

# human
merge_datasets('/content/human_dataset/train/images', '/content/human_dataset/train/labels',
               '/content/combined_dataset/train/images', '/content/combined_dataset/train/labels', prefix="human")
merge_datasets('/content/human_dataset/valid/images', '/content/human_dataset/valid/labels',
               '/content/combined_dataset/valid/images', '/content/combined_dataset/valid/labels', prefix="human")
merge_datasets('/content/human_dataset/test/images', '/content/human_dataset/test/labels',
               '/content/combined_dataset/test/images', '/content/combined_dataset/test/labels', prefix="human")


def list_files(d, exts=('.jpg','.png','.jpeg','.JPG')):
    return [os.path.basename(p) for p in glob.glob(os.path.join(d,'*')) if p.lower().endswith(exts)]

def check_labels(lbl_dir):
    problems = []
    stats = {'total_lbl_files':0, 'total_boxes':0, 'tiny_boxes':0}
    for f in os.listdir(lbl_dir):
        if not f.endswith('.txt'): continue
        stats['total_lbl_files'] += 1
        with open(os.path.join(lbl_dir, f), "r") as fh:
            lines = fh.read().strip().splitlines()
        for L in lines:
            parts = L.split()
            if len(parts) != 5:
                problems.append((f, 'bad_line', L))
                continue
            cls,x,y,w,h = map(float, parts)
            stats['total_boxes'] += 1
            if w*h < 0.0005:
                stats['tiny_boxes'] += 1
    return stats, problems[:10]

splits = ["train", "valid", "test"]
for split in splits:
    img_dir = f"/content/combined_dataset/{split}/images"
    lbl_dir = f"/content/combined_dataset/{split}/labels"
    imgs = list_files(img_dir)
    print(f"{split} imgs: {len(imgs)}")
    print(f"{split} label check:", check_labels(lbl_dir))
    print("-"*40)

In [ ]:
!pip install albumentations

* Новый код аугментации

Этот код предназначен для увеличения разнообразия датасета с помощью аугментаций. Каждое изображение и его метка проходят через набор случайных трансформаций, после чего сохраняются новые варианты изображений вместе с обновлёнными координатами bbox.


* bbox_params - обновляет координаты всех bbox после каждой трансформации

* N_AUGS = 1  перебирает все изображения в папке, находит к ним .txt аннотации и применяет аугментации. Для каждой картинки создаётся N_AUGS новых вариантов - в моем случае (1) - что удваивает мой датасет

* Каждое аугментированное изображение сохраняется в папку out_img_dir.
Его разметка сохраняется в .txt файл в папку out_lbl_dir.
* Если после аугментации боксы исчезли (объект полностью вышел за кадр) - создаётся пустой .txt файл.

# Изменения №2

*Обратная связь по проекту*

Изменения:

* убрал излишний resize
* min_visibility=0.05
* аугментированные копии сохраняются отдельно в train_aug и в дальнейшем к ним идет обращение через конф.файл

In [ ]:
import os
import cv2
from tqdm import tqdm
from albumentations import (
    Compose, HorizontalFlip, CLAHE, RandomBrightnessContrast,
    HueSaturationValue, RandomScale, MotionBlur,
    BboxParams
)


N_AUGS = 1


aug = Compose(
    [
        HorizontalFlip(p=0.5),
        CLAHE(clip_limit=4.0, p=0.3),
        RandomBrightnessContrast(p=0.5),
        HueSaturationValue(hue_shift_limit=15, sat_shift_limit=20, val_shift_limit=15, p=0.5),
        RandomScale(scale_limit=(-0.2, 0.6), p=0.4),
        MotionBlur(blur_limit=7, p=0.3),
    ],
    bbox_params=BboxParams(
        format='yolo',
        min_visibility=0.05,
        clip=True,
        label_fields=['class_labels']
    )
)


input_img_dir = "/content/combined_dataset/train/images"
input_lbl_dir = "/content/combined_dataset/train/labels"
out_img_dir   = "/content/combined_dataset/train_aug/images"
out_lbl_dir   = "/content/combined_dataset/train_aug/labels"

os.makedirs(out_img_dir, exist_ok=True)
os.makedirs(out_lbl_dir, exist_ok=True)

valid_ext = ('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff')


def augment_single_image(img_path, label_path, out_img_dir, out_lbl_dir, n_augs):
    img = cv2.imread(img_path)
    if img is None:
        print(f"Не удалось прочитать {img_path}")
        return

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


    bboxes, class_ids = [], []
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            lines = f.readlines()
        for line in lines:
            parts = line.strip().split()
            if len(parts) == 5:
                class_id, x, y, w, h = map(float, parts)
                bboxes.append([x, y, w, h])
                class_ids.append(int(class_id))

    base_name, ext = os.path.splitext(os.path.basename(img_path))

    for i in range(1, n_augs + 1):
        if bboxes:
            augmented = aug(image=img_rgb, bboxes=bboxes, class_labels=class_ids)
            aug_img_rgb = augmented['image']
            aug_bboxes = augmented['bboxes']
            aug_classes = augmented['class_labels']
        else:
            aug_img_rgb, aug_bboxes, aug_classes = img_rgb, [], []


        out_img_name = f"{base_name}_aug{i}.jpg"
        out_lbl_name = f"{base_name}_aug{i}.txt"


        output_img_path = os.path.join(out_img_dir, out_img_name)
        output_lbl_path = os.path.join(out_lbl_dir, out_lbl_name)


        cv2.imwrite(output_img_path, cv2.cvtColor(aug_img_rgb, cv2.COLOR_RGB2BGR))


        if aug_bboxes:
            with open(output_lbl_path, 'w') as f:
                for cls, (x, y, w, h) in zip(aug_classes, aug_bboxes):
                    f.write(f"{cls} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")
        else:
            open(output_lbl_path, 'w').close()



for img_name in tqdm(os.listdir(input_img_dir), desc="Augmenting"):
    if img_name.lower().endswith(valid_ext):
        img_path = os.path.join(input_img_dir, img_name)
        label_path = os.path.join(input_lbl_dir, os.path.splitext(img_name)[0] + '.txt')
        augment_single_image(img_path, label_path, out_img_dir, out_lbl_dir, n_augs=N_AUGS)


num_imgs = len([f for f in os.listdir(out_img_dir) if f.lower().endswith(valid_ext)])
num_lbls = len([f for f in os.listdir(out_lbl_dir) if f.endswith('.txt')])

print(f"Аугментация завершена!")
print(f"Создано изображений: {num_imgs}")
print(f"Создано лейблов: {num_lbls}")


Проверяю количество данных. в общем выходит 10.109 изображений

In [ ]:
print("train:", len(os.listdir("/content/combined_dataset/train/images")))
print("train_aug:", len(os.listdir("/content/combined_dataset/train_aug/images")))
print("valid:", len(os.listdir("/content/combined_dataset/valid/images")))
print("test:", len(os.listdir("/content/combined_dataset/test/images")))

* Подготовка конфигурационного файла

In [ ]:
data_yaml = """
train:
  - /content/combined_dataset/train/images
  - /content/combined_dataset/train_aug/images

val: /content/combined_dataset/valid/images
test: /content/combined_dataset/test/images

nc: 1
names: ['person']

"""

with open("/content/data.yaml", "w") as f:
    f.write(data_yaml)

In [ ]:
!pip install ultralytics

* 3 обучение YOLOv8s.

In [ ]:
SAVE_PATH = '/content/drive/MyDrive/yolo_training'

from ultralytics import YOLO

model = YOLO('yolov8s.pt')
model.train(
    data='/content/data.yaml',
    epochs=120,
    batch=8,
    imgsz=640,
    optimizer='AdamW',
    hsv_h=0.015,
    mosaic=0.5,
    name='DetectModelS',
    project=SAVE_PATH,
    save_period=50
)

По результатам обучения наблюдается рост метрик

* Precision: 0.76 → 0.83 (модель реже делает ложные срабатывания).

* Recall: 0.50 → 0.54 (чуть лучше находит людей, но прирост небольшой).

* mAP@50: 0.60 → 0.66 (общее качество детекции выросло).

* mAP@50-95: 0.36 → 0.41 (модель стабильнее работает на разных IoU порогах).

Вывод:

Аугментация и переработка датасета сработали и модель стала аккуратнее и внимательнее.

Основной рывок в mAP благодаря тому что данные стали разнообразнее.

Recall не получил сильного прироста поскольку требует датасет свыше 10к изображений.

# New experiments

*Обратная связь по проекту. Последние обучения 1/2.*

Изменения:
* большинство параметров yolo оставляю по умолчанию.
* Добавляю patience
* Используется Collab Pro обучение на гп А100 с увеличенным объемом ОЗУ
* imgsz 640

In [ ]:
SAVE_PATH = '/content/drive/MyDrive/yolo_training'

from ultralytics import YOLO

model = YOLO('yolov8m.pt')
model.train(
    data='/content/data.yaml',
    epochs=200,
    batch=32,
    name='UpdatedModel',
    project=SAVE_PATH,
    save_period=50,
    patience=30
)

Результаты валидации:
* Precision всех классов: 0.85
* Recall всех классов: 0.58
* mAP@50: 0.69
* mAP@50-95: 0.45

Рост метрик засчет исправленой ошибки на этапе комбинирования датасета + изменение кода аугментации + стандартные параметры обучения Yolo.

Обучение завершилось на 117 эпохе (Patience)

In [ ]:
metrics = model.val()


print("Результаты валидации:")
print(f"Precision всех классов: {metrics.box.p.mean():.2f}")
print(f"Recall всех классов: {metrics.box.r.mean():.2f}")
print(f"mAP@50: {metrics.box.map50:.2f}")
print(f"mAP@50-95: {metrics.box.map:.2f}")

# Последнее обучение

*Обратная связь по проекту. Обучение 2/2.*

* patience=40
* imgsz=960

In [ ]:
SAVE_PATH = '/content/drive/MyDrive/yolo_training'

from ultralytics import YOLO

model = YOLO('yolov8m.pt')
model.train(
    data='/content/data.yaml',
    epochs=200,
    imgsz=960,
    batch=16,
    name='SecondUpdatedModel',
    project=SAVE_PATH,
    patience=40
)

Результаты валидации:
* Precision всех классов: 0.85 (без изменений)
* Recall всех классов: 0.65 > 0.58
* mAP@50: 0.76 > 0.69
* mAP@50-95: 0.51 > 0.45


обучение завершилось на 51 эпохе - patience=40

In [ ]:
metrics = model.val()


print("Результаты валидации:")
print(f"Precision всех классов: {metrics.box.p.mean():.2f}")
print(f"Recall всех классов: {metrics.box.r.mean():.2f}")
print(f"mAP@50: {metrics.box.map50:.2f}")
print(f"mAP@50-95: {metrics.box.map:.2f}")

* Вывод графиков

In [ ]:
from IPython.display import Image, display
import os

results_path = '/content/drive/MyDrive/yolo_training/DetectModelS3'

for fname in os.listdir(results_path):
    if fname.endswith('.png'):
        print(fname)
        display(Image(os.path.join(results_path, fname)))

In [ ]:
!pip install ultralytics

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp /content/drive/MyDrive/FinalProject/notebooks/DetectModelS3best.pt .

In [ ]:
!cp /content/drive/MyDrive/yolo_training/SecondUpdatedModel/weights/best.pt .

In [ ]:
!pip install -U python-telegram-bot==22.3

In [ ]:
import os
import cv2
import asyncio
import nest_asyncio
from ultralytics import YOLO
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, ContextTypes, filters

Использую библиотеки для создания телеграм бота

nest_asyncio позволяет "поверх" существующего цикла запускать новые асинхронные процессы, поэтому бот может работать в среде

Логика работы:


Пользователь пишет команду /start - функция start() отправляет приветственное сообщение.

Пользователь отправляет видео - функция handle_video():

* сохраняет видео локально,

* открывает его через OpenCV,

* проходит по каждому кадру,

* прогоняет кадр через YOLO - модель находит объекты,

* считает количество людей (cls == 0),

* рисует рамки и число людей на кадре,

* записывает результат в новое видео.

Когда обработка заканчивается:

* итоговое видео отправляется обратно пользователю,

* временные файлы удаляются, чтобы не захламлять память

In [ ]:
nest_asyncio.apply()

TOKEN = "8341705487:AAEyJCy1Jq0FIeW6wTPtZO3w5H1JDq0AsgY"
WEIGHTS_PATH = "best.pt"
OUTPUT_VIDEO = "output.mp4"


model = YOLO(WEIGHTS_PATH)


async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text(
        "Приветствую. Текущая модель нейросети SecondUpdatedModel.\n"
        "Отправьте видеофайл для проверки"
    )


async def handle_video(update: Update, context: ContextTypes.DEFAULT_TYPE):
    video = update.message.video or update.message.document
    if not video:
        await update.message.reply_text("Пожалуйста, отправьте видеофайл")
        return


    input_path = "input.mp4"
    file = await video.get_file()
    await file.download_to_drive(input_path)


    cap = cv2.VideoCapture(input_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS) or 30
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(OUTPUT_VIDEO, fourcc, fps, (width, height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break


        results = model(frame, conf=0.5)
        count = sum(1 for box in results[0].boxes if int(box.cls[0]) == 0)


        annotated_frame = results[0].plot()
        cv2.putText(annotated_frame, f"People: {count}",
                    (20, 40), cv2.FONT_HERSHEY_SIMPLEX,
                    1, (0, 255, 0), 2)
        out.write(annotated_frame)

    cap.release()
    out.release()


    with open(OUTPUT_VIDEO, "rb") as f:
        await update.message.reply_video(f)


    os.remove(input_path)
    os.remove(OUTPUT_VIDEO)


# Запуск бота
app = Application.builder().token(TOKEN).build()
app.add_handler(CommandHandler("start", start))
app.add_handler(MessageHandler(filters.VIDEO | filters.Document.VIDEO, handle_video))

await app.run_polling()